In [1]:
import pandas as pd
import numpy as np
import os 
import cv2
from skimage.io import ImageCollection
from skimage.io import imread,imshow
from utils_detector import DataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

/home/wvillegas/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
dataset = pd.read_csv(os.path.join('/home','wvillegas','dataset-mask', 'full_masks.csv'))

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['orig'],dataset['mask'],
                                                    test_size = 0.2, random_state=1)

In [5]:
partition = {'train':list(X_train),
             'test': list(X_test)}

In [6]:
img_list = list(X_train) + list(X_test)
mask_list = list(Y_train) + list(Y_test)

In [7]:
labels = dict(zip(img_list, mask_list))

In [8]:
masks_path = os.path.join('/home','wvillegas','dataset-mask','masks')
img_path = os.path.join('/home','wvillegas','dataset-mask','images')

In [9]:
batch_size = 1
dim_img = (1728, 2304)
dim_mask = (658,874)

In [10]:
train_generator = DataGenerator(batch_size=batch_size,dim=dim_img,dim_label=dim_mask,img_path=img_path,
                                labels=labels,list_IDs=partition['train'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)
valid_generator = DataGenerator(batch_size=batch_size,dim=dim_img,dim_label=dim_mask,img_path=img_path,
                                labels=labels,list_IDs=partition['test'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)

In [11]:
# Xtrain = []
# Xtest = []
# Ytrain = []
# Ytest = []
# for i in (X_train.index):
#     Xtrain.append(imread(img_path + '/' + X_train[i]))
#     Ytrain.append(imread(masks_path + '/' + Y_train[i]))
# for i in (X_test.index):
#     Xtest.append(imread(img_path + '/' + X_test[i]))
#     Ytest.append(imread(masks_path + '/' + Y_test[i]))

In [12]:
# for i in np.arange(num_samples):
#     X[i] = cv2.resize(X[i],(0,0), fx=0.5, fy=0.5)
#     Y[i] = cv2.resize(Y[i],(874,658))    
#     Y[i] = cv2.cvtColor(Y[i], cv2.COLOR_BGR2GRAY)
#     Y[i] = Y[i].astype(bool).astype(int)
#     Y[i] = np.expand_dims(Y[i], axis=2)

In [13]:
# imshow(X[0])

In [14]:
# imshow(Y[0][:,:,0])

In [15]:
from keras.applications import MobileNet
from keras.layers import Conv2DTranspose,Conv2D
from keras import Model

In [16]:
mobilenet = MobileNet(include_top=False,weights='imagenet')

/home/wvillegas/miniconda3/lib/python3.6/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [17]:
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [18]:
# t1 = Conv2DTranspose(1,activation='relu',kernel_size=4)(mobilenet.output)
# t2 = Conv2DTranspose(1,activation='relu',kernel_size=8)(t1)
# t3 = Conv2DTranspose(1,activation='relu',kernel_size=16)(t2)

# fin = Conv2D(1,1,activation='sigmoid')(t3)

In [19]:
# test arquitectura paper de FCN 
# deconv1 = Conv2DTranspose(filters=128,kernel_size=3,strides=2,activation='relu')(mobilenet.output)
# deconv2 = Conv2DTranspose(filters=64,kernel_size=3,strides=2,activation='relu')(deconv1)
deconv3 = Conv2DTranspose(filters=32,kernel_size=3,strides=2,activation='relu')(mobilenet.output)
deconv4 = Conv2DTranspose(filters=16,kernel_size=3,strides=2,activation='relu')(deconv3)
deconv5 = Conv2DTranspose(filters=8,kernel_size=4,strides=3,activation='relu')(deconv4)
final = Conv2D(1,1,activation='sigmoid')(deconv5)

In [20]:
fcn = Model(inputs=mobilenet.input,outputs=final)

In [21]:
# fcn.summary()

In [22]:
# for layer in fcn.layers:
#     layer.trainable = False

In [23]:
# for layer in fcn.layers:
#     print(layer.trainable)

In [24]:
# plt.imshow(pred[0,:,:,0], cmap="gray")

In [25]:
# Xtrain = np.asarray(Xtrain)
# Ytrain = np.asarray(Ytrain)
# Xtest = np.asarray(Xtest)
# Ytest = np.asarray(Ytest)

In [26]:
from keras.callbacks import Callback

In [27]:
class LossAccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get['acc'])
        self.val_acc.append(logs.get['val_acc'])
        

In [28]:
# callback = LossAccHistory()

In [29]:
from keras.optimizers import SGD, RMSprop

In [30]:
rms = SGD(lr=0.01,momentum=0.9)
fcn.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

In [31]:
history = fcn.fit_generator(generator=train_generator,validation_data=valid_generator,
                            use_multiprocessing=True,workers=6, epochs=2)

Epoch 1/2


ResourceExhaustedError: OOM when allocating tensor with shape[1,3,1825,2737] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/SGD/gradients/conv1/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1_pad/Pad, PermConstNHWCToNCHW-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics/acc/Mean_1/_1311 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7644_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add re

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
list_img = partition['test'][2:4]

In [ ]:
inp_img = []
for img in list_img:
    temp = cv2.imread(img_path + '/' + img)
    temp = cv2.resize(temp, (0,0), fx=0.5, fy=0.5)
#   temp = cv2.normalize(temp, temp, 0, 255, cv2.NORM_MINMAX)
    inp_img.append(temp)
    
ground_truth = []
for img in list_img:
    mask = labels[img]
    print(mask)
    temp = cv2.imread(masks_path + '/' + mask)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = temp.astype(bool).astype(int)
    ground_truth.append(temp)


In [ ]:
inp_img = np.asarray(inp_img)
ground_truth = np.asarray(ground_truth)

In [ ]:
inp_img.shape

In [ ]:
pred = fcn.predict(inp_img)

In [ ]:
# img = cv2.imread(img_path + '/0717.jpg')
# img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
# img.shape

In [ ]:
# xx = np.empty((1,img.shape[0],img.shape[1], 3))
# xx[0,] = img

In [ ]:
# pred = fcn.predict(xx)
# pred.shape

In [ ]:
pred.shape

In [ ]:
imshow(pred[1,:,:,0])

In [ ]:
ground_truth.shape

In [ ]:
imshow(ground_truth[1,:,:])

In [ ]:
ground_truth[1,:,:,0].sum()